In [2]:
#!pip install pyproj
#!pip install pyshp

In [2]:
import json
from pyproj import Proj, transform
import googlemaps
import folium
import shapefile #pyshp module
import pandas as pd

inProj = Proj(init = 'epsg:5179 ') 
outProj= Proj(init = 'epsg:4326') #wgs84

In [3]:
#파일 읽기
shape = shapefile.Reader("TL_SPRD_MANAGE.shp", encoding = "euc-kr")
print("\n 1. type : " , shape.shapeType) #3 : polyline     # 1. type :  3

#레코드 가져오기
shapeRecs = shape.shapeRecords()
print("\n 2. len : " , len(shapeRecs))                     # 2. len :  1845

fields_node = [x[0] for x in shape.fields][1:]     #필드 데이터
records_node = [list(x.record) for x in shape.shapeRecords()]       #데이터
load_points  = [s.points for s in shape.shapes()]  #좌표 데이터

#dataframe으로 만들기
record_df = pd.DataFrame(columns = fields_node, data = records_node )
record_df.head(3)


 1. type :  3

 2. len :  1845


,ALWNC_DE,ALWNC_RESN,BSI_INT,ENG_RN,MVMN_DE,MVMN_RESN,MVM_RES_CD,NTFC_DE,OPERT_DE,RBP_CN,RDS_DPN_SE,RDS_MAN_NO,REP_CN,RN,RN_CD,ROAD_BT,ROAD_LT,ROA_CLS_SE,SIG_CD,WDR_RD_CD
0,20090710,서울의 남쪽을 연결하는 도로,20,Nambusunhwan-ro,20141030,직권수정(속성변경),99,20100315,20150326104422,강서구 방화동 434-10,0,3,강남구 수서동 329-21,남부순환로,2000003,41.0,32620.0,2,11680,1
1,20100419,양재천 명칭 인용,20,Yangjaecheon-ro,20141030,기재내용정정,71,20100422,20181107140422,서울특별시 서초구 우면동 23-14(경부고속도로 하부),0,5,서울특별시 강남구 대치동 513(영동대로),양재천로,3121023,20.0,4500.0,3,11680,2
2,20100419,영등포의 동쪽 지역을 의미하는 영동이라는 지명에서 유래,20,Yeongdong-daero,20141030,기재내용정정,71,20100422,20181107140506,서울특별시 강남구 개포동 192-1,0,6,서울특별시 성동구 성수동2가 158-1(영동대교 북단),영동대로,2122002,70.0,5683.0,2,11680,2


NULL = 0  
POINT = 1  
POLYLINE = 3  
POLYGON = 5  
MULTIPOINT = 8  
POINTZ = 11  
POLYLINEZ = 13  
POLYGONZ = 15  
MULTIPOINTZ = 18  
POINTM = 21  
POLYLINEM = 23  
POLYGONM = 25  
MULTIPOINTM = 28  
MULTIPATCH = 31  

## 좌표정보 합치기

In [4]:
#매우느림
coords = []
for i in range( 0, len(load_points)) :
    a = load_points[i]
    b = list(map(lambda x: transform(inProj,outProj,x[0],x[1]), a))
    coords.append(b)


In [5]:
#dataframe에 좌표정보 삽입
record_df['COORDS'] = coords
record_df.head(5)

,ALWNC_DE,ALWNC_RESN,BSI_INT,ENG_RN,MVMN_DE,MVMN_RESN,MVM_RES_CD,NTFC_DE,OPERT_DE,RBP_CN,...,RDS_MAN_NO,REP_CN,RN,RN_CD,ROAD_BT,ROAD_LT,ROA_CLS_SE,SIG_CD,WDR_RD_CD,COORDS
0,20090710,서울의 남쪽을 연결하는 도로,20,Nambusunhwan-ro,20141030,직권수정(속성변경),99,20100315,20150326104422,강서구 방화동 434-10,...,3,강남구 수서동 329-21,남부순환로,2000003,41.0,32620.0,2,11680,1,"[(127.03401035390544, 37.48454726866469), (127..."
1,20100419,양재천 명칭 인용,20,Yangjaecheon-ro,20141030,기재내용정정,71,20100422,20181107140422,서울특별시 서초구 우면동 23-14(경부고속도로 하부),...,5,서울특별시 강남구 대치동 513(영동대로),양재천로,3121023,20.0,4500.0,3,11680,2,"[(127.04359887908493, 37.48039823968481), (127..."
2,20100419,영등포의 동쪽 지역을 의미하는 영동이라는 지명에서 유래,20,Yeongdong-daero,20141030,기재내용정정,71,20100422,20181107140506,서울특별시 강남구 개포동 192-1,...,6,서울특별시 성동구 성수동2가 158-1(영동대교 북단),영동대로,2122002,70.0,5683.0,2,11680,2,"[(127.07554544262126, 37.486157582444775), (12..."
3,20100419,본래 학로였으나 1984년 11월 7일 가로명 제정때 학동로로 부여,20,Hakdong-ro,20141030,"광역도로 정비 및 부여사유, 부여방식 정비",90,20100422,20141030225537,논현동 50-2,...,7,청담동 134-18,학동로,3122011,40.0,3755.0,3,11680,3,"[(127.02132893005424, 37.51110040134589), (127..."
4,20080901,이 노선이 지나는 일원동의 행정구역명을 적용하여 제정,20,Irwon-ro,20141030,"광역도로 정비 및 부여사유, 부여방식 정비",90,20101203,20141030225537,일원동 618-3,...,8,일원동 738,일원로,3122009,25.0,1430.0,3,11680,3,"[(127.07952543863435, 37.493647396353744), (12..."


In [6]:
record_df.to_csv("1_1_all_records_raw.csv", mode='w', encoding='ms949') # edge(=link) list

## 좌표 분리 (하나씩 꺼내기)

In [7]:
#원하는 지역만 선정하기
df = record_df[(record_df.RBP_CN.str.contains('역삼동')) | (record_df.REP_CN.str.contains('역삼동')) ]
df = df.reset_index(drop=True)
df.head(10)
df.to_csv("1_2_역삼동_all_records_raw.csv", mode='w', encoding='ms949') # edge(=link) list

In [8]:
all_coords = pd.DataFrame(columns = ['SIG_CD', 'RDS_MAN_NO', 'RN', 'RN_CD', 'ROAD_BT', 'ROAD_LT' , 'LAT', 'LNG'])
tmp = pd.DataFrame(columns =['SIG_CD', 'RDS_MAN_NO', 'RN', 'RN_CD' , 'RDS_MAN_NO2', 'ROAD_BT', 'ROAD_LT' , 'COORDS'])                                                       
    
for i in range( 0, len(df)):
    tmp = df.loc[i, ['SIG_CD', 'RDS_MAN_NO', 'RN', 'RN_CD', 'ROAD_BT', 'ROAD_LT' , 'COORDS']]
    for j in range(0, len(tmp.COORDS)) :
        all_coords = all_coords.append(all_coords.from_dict({'SIG_CD' : tmp.SIG_CD ,'RDS_MAN_NO':tmp.RDS_MAN_NO
                                                             , 'RN': tmp.RN, 'RN_CD' : tmp.RN_CD, 'RDS_MAN_NO2' :  j
                                                             , 'ROAD_BT':tmp.ROAD_BT, 'ROAD_LT' : tmp.ROAD_LT
                                                             , 'LAT': tmp.COORDS[j][1], 'LNG' : tmp.COORDS[j][0]
                                                            } , orient = 'index').T)


C:\anaconda\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [9]:
all_coords = all_coords.sort_values(by=['RDS_MAN_NO','RDS_MAN_NO2']).reset_index(drop=True)
all_coords #2036개

# KEY : SIG_CD, RDS_MAN_NO,RDS_MAN_NO2
all_coords.to_csv("1_3_COORDS_IN_YS.csv", mode='w', encoding='ms949') # edge(=link) list

In [10]:
a = all_coords.loc[:,[ 'SIG_CD', 'RDS_MAN_NO', 'RDS_MAN_NO2', 'RN_CD', 'RN' ,'LAT' , 'LNG'] ]
#a.loc[8:11,'LAT'].values
a

,SIG_CD,RDS_MAN_NO,RDS_MAN_NO2,RN_CD,RN,LAT,LNG
0,11680,12,0,3122008,역삼로,37.493,127.03
1,11680,12,1,3122008,역삼로,37.4931,127.03
2,11680,12,2,3122008,역삼로,37.4933,127.031
3,11680,12,3,3122008,역삼로,37.4936,127.032
4,11680,12,4,3122008,역삼로,37.4938,127.033
5,11680,12,5,3122008,역삼로,37.4939,127.033
6,11680,12,6,3122008,역삼로,37.4939,127.033
7,11680,12,7,3122008,역삼로,37.494,127.033
8,11680,12,8,3122008,역삼로,37.4941,127.033
9,11680,12,9,3122008,역삼로,37.4941,127.033


In [11]:
to_map1 = folium.Map(location= [a.loc[0,'LAT'], a.loc[0,'LNG']]  , zoom_start=12)

In [12]:
for i in range(0, len(a) ) :
    folium.CircleMarker([a.loc[i,'LAT'], a.loc[i,'LNG']], radius = 3).add_to(to_map1)        
to_map1

In [15]:
to_map2 = folium.Map(location= [a.loc[0,'LAT'], a.loc[0,'LNG']]  , zoom_start=12)
for i in range(0, len(a) ) :
    folium.CircleMarker([a.loc[i,'LAT'], a.loc[i,'LNG']], radius = 3).add_to(to_map2)        
to_map2